
# HMM con SWIG y Python

Este notebook implementa un **Modelo Oculto de Markov (HMM)** en C++ con librería dinámica usando **SWIG**,
y una versión en Python puro. Se comparan ambas implementaciones en cuanto a funcionalidad y tiempos de ejecución.

Incluye las funciones requeridas por la rúbrica:
- **Evaluación** (Forward): calcular la probabilidad de una secuencia.
- **Reconocimiento** (Viterbi): encontrar la ruta más probable (H/L).

Realizado por:
- Galvan Sierra David
- Lopez Copete Ferney
- Trejos Pamplona Leidy Melissa

In [1]:

# Instalación de dependencias necesarias (ejecutar en Colab)
!apt-get update -qq
!apt-get install -y swig g++ build-essential python3-dev -qq
!python3 --version
!swig -version


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package swig4.0.
(Reading database ... 126435 files and directories currently installed.)
Preparing to unpack .../swig4.0_4.0.2-1ubuntu1_amd64.deb ...
Unpacking swig4.0 (4.0.2-1ubuntu1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_4.0.2-1ubuntu1_all.deb ...
Unpacking swig (4.0.2-1ubuntu1) ...
Setting up swig4.0 (4.0.2-1ubuntu1) ...
Setting up swig (4.0.2-1ubuntu1) ...
Processing triggers for man-db (2.10.2-1) ...
Python 3.12.11

SWIG Version 4.0.2

Compiled with g++ [x86_64-pc-linux-gnu]

Configured options: +pcre

Please see http://www.swig.org for reporting bugs and further information


In [2]:

%%bash
cat > hmm.h <<'EOF'
#ifndef HMM_H
#define HMM_H

#include <vector>
#include <string>

class HMM {
public:
    HMM();
    double evaluate_log(const std::string &seq) const;
    double evaluate_prob(const std::string &seq) const;
    std::string viterbi_path(const std::string &seq) const;
private:
    int nuc_idx(char c) const;
    std::vector<std::vector<double>> A;
    std::vector<std::vector<double>> B;
    std::vector<double> pi;
};

#endif // HMM_H
EOF


In [3]:

%%bash
cat > hmm.cpp <<'EOF'
#include "hmm.h"
#include <cmath>
#include <limits>
#include <algorithm>

HMM::HMM() {
    A = std::vector<std::vector<double>>(2, std::vector<double>(2, 0.0));
    A[0][0] = 0.5; A[0][1] = 0.5;
    A[1][0] = 0.4; A[1][1] = 0.6;

    B = std::vector<std::vector<double>>(2, std::vector<double>(4, 0.0));
    B[0][0] = 0.2; B[0][1] = 0.3; B[0][2] = 0.3; B[0][3] = 0.2;
    B[1][0] = 0.3; B[1][1] = 0.2; B[1][2] = 0.2; B[1][3] = 0.3;

    pi = std::vector<double>(2, 0.5);
}

int HMM::nuc_idx(char c) const {
    switch (c) {
        case 'A': case 'a': return 0;
        case 'C': case 'c': return 1;
        case 'G': case 'g': return 2;
        case 'T': case 't': return 3;
        default: return 0;
    }
}

static double log_sum_exp(double x, double y) {
    if (x == -std::numeric_limits<double>::infinity()) return y;
    if (y == -std::numeric_limits<double>::infinity()) return x;
    if (x < y) std::swap(x, y);
    return x + std::log1p(std::exp(y - x));
}

double HMM::evaluate_log(const std::string &seq) const {
    size_t T = seq.size();
    if (T == 0) return -std::numeric_limits<double>::infinity();
    std::vector<double> log_pi(2);
    std::vector<std::vector<double>> log_A(2, std::vector<double>(2));
    std::vector<std::vector<double>> log_B(2, std::vector<double>(4));
    for (int i=0;i<2;++i){
        log_pi[i] = std::log(pi[i]);
        for (int j=0;j<2;++j) log_A[i][j] = std::log(A[i][j]);
        for (int k=0;k<4;++k) log_B[i][k] = std::log(B[i][k]);
    }
    std::vector<std::vector<double>> alpha(T, std::vector<double>(2, -std::numeric_limits<double>::infinity()));
    int idx0 = nuc_idx(seq[0]);
    for (int i=0;i<2;++i) alpha[0][i] = log_pi[i] + log_B[i][idx0];
    for (size_t t=1;t<T;++t){
        int idx = nuc_idx(seq[t]);
        for (int j=0;j<2;++j){
            double acc = -std::numeric_limits<double>::infinity();
            for (int i=0;i<2;++i){
                double cand = alpha[t-1][i] + log_A[i][j];
                acc = log_sum_exp(acc, cand);
            }
            alpha[t][j] = acc + log_B[j][idx];
        }
    }
    double log_prob = -std::numeric_limits<double>::infinity();
    for (int i=0;i<2;++i) log_prob = log_sum_exp(log_prob, alpha[T-1][i]);
    return log_prob;
}

double HMM::evaluate_prob(const std::string &seq) const {
    double lp = evaluate_log(seq);
    if (lp < -700.0) return 0.0;
    return std::exp(lp);
}

std::string HMM::viterbi_path(const std::string &seq) const {
    size_t T = seq.size();
    if (T==0) return "";
    std::vector<double> log_pi(2);
    std::vector<std::vector<double>> log_A(2, std::vector<double>(2));
    std::vector<std::vector<double>> log_B(2, std::vector<double>(4));
    for (int i=0;i<2;++i){
        log_pi[i] = std::log(pi[i]);
        for (int j=0;j<2;++j) log_A[i][j] = std::log(A[i][j]);
        for (int k=0;k<4;++k) log_B[i][k] = std::log(B[i][k]);
    }
    std::vector<std::vector<double>> delta(T, std::vector<double>(2, -std::numeric_limits<double>::infinity()));
    std::vector<std::vector<int>> psi(T, std::vector<int>(2,0));
    int idx0 = nuc_idx(seq[0]);
    for (int i=0;i<2;++i){ delta[0][i] = log_pi[i] + log_B[i][idx0]; psi[0][i]=0; }
    for (size_t t=1;t<T;++t){
        int idx = nuc_idx(seq[t]);
        for (int j=0;j<2;++j){
            double best_val = -std::numeric_limits<double>::infinity();
            int best_state = 0;
            for (int i=0;i<2;++i){
                double val = delta[t-1][i] + log_A[i][j];
                if (val > best_val){ best_val = val; best_state = i; }
            }
            delta[t][j] = best_val + log_B[j][idx];
            psi[t][j] = best_state;
        }
    }
    std::vector<int> states(T);
    double best_val = -std::numeric_limits<double>::infinity();
    int best_state = 0;
    for (int i=0;i<2;++i) if (delta[T-1][i] > best_val){ best_val = delta[T-1][i]; best_state = i; }
    states[T-1] = best_state;
    for (int t=T-1;t>=1;--t) states[t-1] = psi[t][states[t]];
    std::string path; path.reserve(T);
    for (size_t t=0;t<T;++t) path.push_back(states[t]==0?'H':'L');
    return path;
}
EOF


In [4]:

%%bash
cat > hmm.i <<'EOF'
%module hmm
%{
#include "hmm.h"
%}
%include <std_string.i>
%include "hmm.h"
EOF


In [5]:
%%bash
swig -c++ -python hmm.i
PY_INC=$(python3 -c "import sysconfig; print(sysconfig.get_path('include'))")
g++ -fPIC -std=c++11 -c hmm.cpp hmm_wrap.cxx -I${PY_INC}
g++ -shared hmm.o hmm_wrap.o -o _hmm.so
ls -l _hmm.so


-rwxr-xr-x 1 root root 172440 Sep 24 01:35 _hmm.so


In [6]:
import importlib, sys
sys.path.append('.')
m = importlib.import_module('hmm')
print("SWIG module imported:", hasattr(m, "HMM"))

SWIG module imported: True


In [7]:

%%bash
cat > hmm_pyimpl.py <<'EOF'
import math

class HMM:
    def __init__(self):
        self.A = [[0.5, 0.5], [0.4, 0.6]]
        self.B = [[0.2, 0.3, 0.3, 0.2],
                  [0.3, 0.2, 0.2, 0.3]]
        self.pi = [0.5, 0.5]

    def nuc_idx(self, c: str) -> int:
        c = c.upper()
        return {'A':0,'C':1,'G':2,'T':3}.get(c,0)

    def log_sum_exp(self, x, y):
        if x == -math.inf: return y
        if y == -math.inf: return x
        if x < y: x,y = y,x
        return x + math.log1p(math.exp(y - x))

    def evaluate_log(self, seq: str) -> float:
        T = len(seq)
        if T == 0: return -math.inf
        log_pi = [math.log(p) for p in self.pi]
        log_A = [[math.log(x) for x in row] for row in self.A]
        log_B = [[math.log(x) for x in row] for row in self.B]
        alpha = [[-math.inf]*2 for _ in range(T)]
        idx0 = self.nuc_idx(seq[0])
        for i in range(2): alpha[0][i] = log_pi[i] + log_B[i][idx0]
        for t in range(1,T):
            idx = self.nuc_idx(seq[t])
            for j in range(2):
                acc = -math.inf
                for i in range(2):
                    cand = alpha[t-1][i] + log_A[i][j]
                    acc = self.log_sum_exp(acc, cand)
                alpha[t][j] = acc + log_B[j][idx]
        logp = -math.inf
        for i in range(2): logp = self.log_sum_exp(logp, alpha[T-1][i])
        return logp

    def viterbi_path(self, seq: str) -> str:
        T = len(seq)
        if T == 0: return ''
        log_pi = [math.log(p) for p in self.pi]
        log_A = [[math.log(x) for x in row] for row in self.A]
        log_B = [[math.log(x) for x in row] for row in self.B]
        delta = [[-math.inf]*2 for _ in range(T)]
        psi = [[0]*2 for _ in range(T)]
        idx0 = self.nuc_idx(seq[0])
        for i in range(2): delta[0][i] = log_pi[i] + log_B[i][idx0]
        for t in range(1,T):
            idx = self.nuc_idx(seq[t])
            for j in range(2):
                best_val = -math.inf; best_state = 0
                for i in range(2):
                    val = delta[t-1][i] + log_A[i][j]
                    if val > best_val: best_val = val; best_state = i
                delta[t][j] = best_val + log_B[j][idx]
                psi[t][j] = best_state
        states = [0]*T
        best_val = -math.inf; best_state = 0
        for i in range(2):
            if delta[T-1][i] > best_val: best_val = delta[T-1][i]; best_state = i
        states[T-1] = best_state
        for t in range(T-1,0,-1):
            states[t-1] = psi[t][states[t]]
        path = ''.join('H' if s==0 else 'L' for s in states)
        return path
EOF


In [21]:

# Prueba rápida
import sys, importlib
import math

sys.path.append('.')
import hmm_pyimpl
model_py = hmm_pyimpl.HMM()
seq = "GGCACTGAA"
print("Python -> logp:", math.exp(model_py.evaluate_log(seq)), "path:", model_py.viterbi_path(seq))
try:
    hmm = importlib.import_module('hmm')
    model_cpp = hmm.HMM()
    print("C++ -> logp:", math.exp(model_cpp.evaluate_log(seq)), "path:", model_cpp.viterbi_path(seq))
except:
    print("C++ no disponible")


Python -> logp: 3.7910160355200046e-06 path: HHHLLLLLL
C++ -> logp: 3.7910160355200046e-06 path: HHHLLLLLL


In [16]:
# Prueba rápida con salida embellecida
import sys, importlib
import math

sys.path.append('.')
import hmm_pyimpl

def format_probability(log_prob):
    """Formatea la probabilidad de manera legible evitando muchos ceros"""
    prob = math.exp(log_prob)

    # Si la probabilidad es muy pequeña, usar notación científica
    if prob < 0.001:
        return f"{prob:.4e}"
    # Si es razonablemente grande, usar formato decimal con precisión adecuada
    elif prob < 0.1:
        return f"{prob:.6f}"
    else:
        return f"{prob:.4f}"

model_py = hmm_pyimpl.HMM()
seq = "GGCA"

log_prob_py = model_py.evaluate_log(seq)
path_py = model_py.viterbi_path(seq)

print(f"Python -> logp: {format_probability(log_prob_py)} (log: {log_prob_py:.6f}), path: {path_py}")

try:
    hmm = importlib.import_module('hmm')
    model_cpp = hmm.HMM()

    log_prob_cpp = model_cpp.evaluate_log(seq)
    path_cpp = model_cpp.viterbi_path(seq)

    print(f"C++    -> logp: {format_probability(log_prob_cpp)} (log: {log_prob_cpp:.6f}), path: {path_cpp}")

    # Comparación adicional
    diff_log = abs(log_prob_py - log_prob_cpp)
    diff_prob = abs(math.exp(log_prob_py) - math.exp(log_prob_cpp))
    path_match = "✓" if path_py == path_cpp else "✗"

    print(f"\nComparación:")
    print(f"  Diferencia log-prob: {diff_log:.2e}")
    print(f"  Diferencia prob:     {diff_prob:.2e}")
    print(f"  Paths coinciden:     {path_match}")

except ImportError:
    print("C++ no disponible")
except Exception as e:
    print(f"Error con C++: {e}")

Python -> logp: 0.003843 (log: -5.561463), path: HHHL
C++    -> logp: 0.003843 (log: -5.561463), path: HHHL

Comparación:
  Diferencia log-prob: 0.00e+00
  Diferencia prob:     0.00e+00
  Paths coinciden:     ✓


In [19]:

# Benchmark
import time, random
seq_long = ''.join(random.choice('ACGT') for _ in range(500000))
N=5
t0=time.perf_counter()
for _ in range(N): model_py.evaluate_log(seq_long)
t1=time.perf_counter()
print("Python total:", t1-t0, " segundos")
try:
    model_cpp=hmm.HMM()
    t0=time.perf_counter()
    for _ in range(N): model_cpp.evaluate_log(seq_long)
    t1=time.perf_counter()
    print("C++ total:", t1-t0, " segundos")
except:
    print("C++ no disponible")


Python total: 7.988330207999979  segundos
C++ total: 1.273984548000044  segundos


In [20]:
# Benchmark completo con probabilidad y decodificación
import time, random, math
import sys, importlib

# Importar modelo Python
sys.path.append('.')
import hmm_pyimpl
model_py = hmm_pyimpl.HMM()

# Generar secuencia larga para benchmark
seq_long = ''.join(random.choice('ACGT') for _ in range(500000))
N = 5

print("=== BENCHMARK DE RENDIMIENTO ===")
print(f"Secuencia de longitud: {len(seq_long)}")
print(f"Número de repeticiones: {N}")
print()

# Benchmark Python
print("--- Python Implementation ---")
t0 = time.perf_counter()
for _ in range(N):
    model_py.evaluate_log(seq_long)
t1 = time.perf_counter()
python_time = t1 - t0
print(f"Python total: {python_time:.4f} segundos")
print(f"Python promedio por ejecución: {python_time/N:.4f} segundos")

# Benchmark C++ (si está disponible)
print("\n--- C++ Implementation ---")
try:
    hmm = importlib.import_module('hmm')
    model_cpp = hmm.HMM()

    t0 = time.perf_counter()
    for _ in range(N):
        model_cpp.evaluate_log(seq_long)
    t1 = time.perf_counter()
    cpp_time = t1 - t0
    print(f"C++ total: {cpp_time:.4f} segundos")
    print(f"C++ promedio por ejecución: {cpp_time/N:.4f} segundos")
    print(f"Speedup: {python_time/cpp_time:.2f}x más rápido")
    cpp_available = True
except:
    print("C++ no disponible")
    cpp_available = False

print("\n" + "="*50)

# Prueba de funcionalidad con secuencia corta
print("=== PRUEBA DE FUNCIONALIDAD ===")
seq_test = "GGCA"
print(f"Secuencia de prueba: {seq_test}")
print()

# Prueba Python
print("--- Python Results ---")
logp_py = model_py.evaluate_log(seq_test)
prob_py = math.exp(logp_py)
path_py = model_py.viterbi_path(seq_test)
print(f"Log-probabilidad: {logp_py}")
print(f"Probabilidad: {prob_py:.8f}")
print(f"Camino Viterbi: {path_py}")

# Prueba C++ (si está disponible)
if cpp_available:
    print("\n--- C++ Results ---")
    logp_cpp = model_cpp.evaluate_log(seq_test)
    prob_cpp = math.exp(logp_cpp)
    path_cpp = model_cpp.viterbi_path(seq_test)
    print(f"Log-probabilidad: {logp_cpp}")
    print(f"Probabilidad: {prob_cpp:.8f}")
    print(f"Camino Viterbi: {path_cpp}")

    # Verificar consistencia
    print("\n--- Verificación de Consistencia ---")
    prob_diff = abs(prob_py - prob_cpp)
    path_match = path_py == path_cpp
    print(f"Diferencia en probabilidad: {prob_diff:.2e}")
    print(f"Caminos coinciden: {path_match}")

    if prob_diff < 1e-10 and path_match:
        print("✓ Implementaciones consistentes")
    else:
        print("✗ Posibles discrepancias entre implementaciones")

print("\n" + "="*50)
print("Benchmark completado")

=== BENCHMARK DE RENDIMIENTO ===
Secuencia de longitud: 500000
Número de repeticiones: 5

--- Python Implementation ---
Python total: 7.9849 segundos
Python promedio por ejecución: 1.5970 segundos

--- C++ Implementation ---
C++ total: 1.2222 segundos
C++ promedio por ejecución: 0.2444 segundos
Speedup: 6.53x más rápido

=== PRUEBA DE FUNCIONALIDAD ===
Secuencia de prueba: GGCA

--- Python Results ---
Log-probabilidad: -5.561462936154914
Probabilidad: 0.00384315
Camino Viterbi: HHHL

--- C++ Results ---
Log-probabilidad: -5.561462936154914
Probabilidad: 0.00384315
Camino Viterbi: HHHL

--- Verificación de Consistencia ---
Diferencia en probabilidad: 0.00e+00
Caminos coinciden: True
✓ Implementaciones consistentes

Benchmark completado


In [10]:

# Helper regiones H
def regions_from_path(path: str):
    regions=[]; in_reg=False; start=0
    for i,ch in enumerate(path):
        if ch=='H' and not in_reg:
            in_reg=True; start=i
        elif ch!='H' and in_reg:
            in_reg=False; regions.append((start,i-1))
    if in_reg: regions.append((start,len(path)-1))
    return regions

print("Regiones H:", regions_from_path(model_py.viterbi_path("GGCACTGAA")))


Regiones H: [(0, 2)]


In [23]:
# Debug detallado del algoritmo de Viterbi
import math

class HMM_Debug:
    def __init__(self):
        self.A = [[0.5, 0.5], [0.4, 0.6]]
        self.B = [[0.2, 0.3, 0.3, 0.2],   # Estado 0 (H): favorece C,G
                  [0.3, 0.2, 0.2, 0.3]]   # Estado 1 (L): favorece A,T
        self.pi = [0.5, 0.5]

    def nuc_idx(self, c: str) -> int:
        c = c.upper()
        return {'A':0,'C':1,'G':2,'T':3}.get(c,0)

    def viterbi_path_debug(self, seq: str) -> str:
        """
        Implementación con debug detallado
        """
        print(f"=== DEBUG VITERBI PARA SECUENCIA: {seq} ===")
        T = len(seq)
        if T == 0: return ''

        # Convertir a log-espacio
        log_pi = [math.log(p) for p in self.pi]
        log_A = [[math.log(x) for x in row] for row in self.A]
        log_B = [[math.log(x) for x in row] for row in self.B]

        print("Parámetros del modelo:")
        print("A (transiciones):", self.A)
        print("B (emisiones):", self.B)
        print("π (inicial):", self.pi)
        print()

        # Inicializar matrices
        delta = [[-math.inf]*2 for _ in range(T)]
        psi = [[0]*2 for _ in range(T)]

        # Inicialización t=0
        idx0 = self.nuc_idx(seq[0])
        print(f"t=0: {seq[0]} (índice={idx0})")
        for i in range(2):
            delta[0][i] = log_pi[i] + log_B[i][idx0]
            print(f"  δ[0][{i}] = log({self.pi[i]}) + log({self.B[i][idx0]}) = {delta[0][i]:.4f}")
        print()

        # Recursión
        for t in range(1, T):
            idx = self.nuc_idx(seq[t])
            print(f"t={t}: {seq[t]} (índice={idx})")

            for j in range(2):
                best_val = -math.inf
                best_state = 0
                print(f"  Para estado {j}:")

                for i in range(2):
                    val = delta[t-1][i] + log_A[i][j]
                    print(f"    Desde estado {i}: {delta[t-1][i]:.4f} + log({self.A[i][j]}) = {val:.4f}")
                    if val > best_val:
                        best_val = val
                        best_state = i

                delta[t][j] = best_val + log_B[j][idx]
                psi[t][j] = best_state
                print(f"    Mejor: estado {best_state}, δ[{t}][{j}] = {best_val:.4f} + log({self.B[j][idx]}) = {delta[t][j]:.4f}")
                print(f"    ψ[{t}][{j}] = {best_state}")
            print()

        # Terminación
        print("Terminación:")
        states = [0]*T
        best_val = -math.inf
        best_state = 0
        for i in range(2):
            print(f"  Estado final {i}: δ[{T-1}][{i}] = {delta[T-1][i]:.4f}")
            if delta[T-1][i] > best_val:
                best_val = delta[T-1][i]
                best_state = i

        states[T-1] = best_state
        print(f"Mejor estado final: {best_state}")
        print()

        # Backtracking
        print("Backtracking:")
        print(f"  states[{T-1}] = {states[T-1]}")
        for t in range(T-1, 0, -1):
            states[t-1] = psi[t][states[t]]
            print(f"  states[{t-1}] = ψ[{t}][{states[t]}] = {states[t-1]}")

        print(f"Secuencia de estados: {states}")
        path = ''.join('H' if s==0 else 'L' for s in states)
        print(f"Camino final: {path}")

        return path

    def viterbi_path_original(self, seq: str) -> str:
        """Implementación original para comparar"""
        T = len(seq)
        if T == 0: return ''
        log_pi = [math.log(p) for p in self.pi]
        log_A = [[math.log(x) for x in row] for row in self.A]
        log_B = [[math.log(x) for x in row] for row in self.B]
        delta = [[-math.inf]*2 for _ in range(T)]
        psi = [[0]*2 for _ in range(T)]
        idx0 = self.nuc_idx(seq[0])
        for i in range(2): delta[0][i] = log_pi[i] + log_B[i][idx0]
        for t in range(1,T):
            idx = self.nuc_idx(seq[t])
            for j in range(2):
                best_val = -math.inf; best_state = 0
                for i in range(2):
                    val = delta[t-1][i] + log_A[i][j]
                    if val > best_val: best_val = val; best_state = i
                delta[t][j] = best_val + log_B[j][idx]
                psi[t][j] = best_state
        states = [0]*T
        best_val = -math.inf; best_state = 0
        for i in range(2):
            if delta[T-1][i] > best_val: best_val = delta[T-1][i]; best_state = i
        states[T-1] = best_state
        for t in range(T-1,0,-1):
            states[t-1] = psi[t][states[t]]
        path = ''.join('H' if s==0 else 'L' for s in states)
        return path

# Probar con la secuencia problemática
hmm_debug = HMM_Debug()
seq_test = "GGCACTGAA"

print("RESULTADO CON DEBUG:")
result_debug = hmm_debug.viterbi_path_debug(seq_test)

print(f"\nRESULTADO ORIGINAL:")
result_original = hmm_debug.viterbi_path_original(seq_test)
print(f"Camino: {result_original}")

print(f"\nCOMPARACIÓN:")
print(f"Obtenido:  {result_original}")
print(f"Esperado:  LLHHHHLLL")
print(f"¿Coinciden? {result_original == 'LLHHHHLLL'}")

# Análisis adicional
print(f"\nANÁLISIS DE PROBABILIDADES DE EMISIÓN:")
print("Para cada nucleótido, ¿qué estado es más probable?")
for i, nuc in enumerate(seq_test):
    idx = hmm_debug.nuc_idx(nuc)
    prob_H = hmm_debug.B[0][idx]  # Probabilidad en estado H
    prob_L = hmm_debug.B[1][idx]  # Probabilidad en estado L
    better = "H" if prob_H > prob_L else "L"
    print(f"  {nuc} (pos {i}): P(H)={prob_H}, P(L)={prob_L} → mejor: {better}")

RESULTADO CON DEBUG:
=== DEBUG VITERBI PARA SECUENCIA: GGCACTGAA ===
Parámetros del modelo:
A (transiciones): [[0.5, 0.5], [0.4, 0.6]]
B (emisiones): [[0.2, 0.3, 0.3, 0.2], [0.3, 0.2, 0.2, 0.3]]
π (inicial): [0.5, 0.5]

t=0: G (índice=2)
  δ[0][0] = log(0.5) + log(0.3) = -1.8971
  δ[0][1] = log(0.5) + log(0.2) = -2.3026

t=1: G (índice=2)
  Para estado 0:
    Desde estado 0: -1.8971 + log(0.5) = -2.5903
    Desde estado 1: -2.3026 + log(0.4) = -3.2189
    Mejor: estado 0, δ[1][0] = -2.5903 + log(0.3) = -3.7942
    ψ[1][0] = 0
  Para estado 1:
    Desde estado 0: -1.8971 + log(0.5) = -2.5903
    Desde estado 1: -2.3026 + log(0.6) = -2.8134
    Mejor: estado 0, δ[1][1] = -2.5903 + log(0.2) = -4.1997
    ψ[1][1] = 0

t=2: C (índice=1)
  Para estado 0:
    Desde estado 0: -3.7942 + log(0.5) = -4.4874
    Desde estado 1: -4.1997 + log(0.4) = -5.1160
    Mejor: estado 0, δ[2][0] = -4.4874 + log(0.3) = -5.6914
    ψ[2][0] = 0
  Para estado 1:
    Desde estado 0: -3.7942 + log(0.5) = -4.4874
 